# Exercise SoSe 2024
We want you to illustrate data. For questions, there are three Q and A session coming up. We recommend taking part in these Q and A sessions and be prepared.

* Results must be visualized
* Describe what you do (code)

**You are not allowed to use data already used in our Jupyter Notebooks!** We provide Links in the Jupyter Notebook for help. Take a look at the resource chapter in Moodle, there you many links, that can help. 

Remember, this is an exam admission task. We expect short, precise implementations. Sections/distances should be chosen so that your computer does not need too long computing times.  

Useful links:
* The actual Map [Open Street Map](https://www.openstreetmap.org/search?query=Aachen#map=12/50.7597/6.0964&layers=D)
* Open Street Map tag information for querying [Wind](https://wiki.openstreetmap.org/wiki/Tag:generator:source%3Dwind), [Water](https://wiki.openstreetmap.org/wiki/Tag:plant:source%3Dhydro), [Solar](https://wiki.openstreetmap.org/wiki/Tag:generator:source=solar?uselang=en), [Geothermal](https://wiki.openstreetmap.org/wiki/Tag:plant:source%3Dgeothermal), [Biomass](https://wiki.openstreetmap.org/wiki/DE:Tag:generator:source%3Dbiomass)


## How to proceed
* Select your favourite spot in Aachen and mark it on an interactive map using an icon of your choice.
* Search for one renewable energy resource position in and around the selected city and visualize the data in your interactive map using the library Python Ipyleaflet using for example an icon. 
* Finally, calculate the shortest path between your favourite spot in Aachen and the renewable energy resource and visualize it as well in your interactive map. 


## Submission
**FINAL Submission 23.06.2024 for your Jupyter Notebook**  
Required for exam admission.
Please submit:  
* **THIS** juypter notebook file filled with your code
* All used source/ created/ data files 

Submit as **ZIP** file, named like: **surname_student-id_course_year** in **RWTHMoodle**.
If you have worked in pairs, please name the file as follows: **surname1_student-id1_surname2_student-id2_course_year**

In [3]:
!pip install -r requirements.txt

Processing /home/conda/feedstock_root/build_artifacts/attrs_1620387926260/work (from -r requirements.txt (line 9))
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/conda/feedstock_root/build_artifacts/attrs_1620387926260/work'



In [5]:
#Import all necessary packages/ libraries 
import folium
import osmnx as ox
import pandas as pd
import geopandas as gpd
import os
import sys
import matplotlib.pyplot as plt 
import networkx as nx
import contextily
from descartes import PolygonPatch
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from IPython.display import Image
from ipyleaflet import Map, GeoData, basemaps, LayersControl
from ipyleaflet import Marker, Icon, Map , Popup
from ipywidgets import HTML
%matplotlib inline
ox.config(log_console=True)
ox.__version__

ImportError: cannot import name 'TopologicalError' from 'shapely.geos' (/opt/conda/lib/python3.9/site-packages/shapely/geos.py)

In [6]:
#Defining walk or building tags for quering OSM for city of Aachen. Also, using the Aachen city center coordinates as a reference point.
#OSM data for all the buildings at a distance of 5km from city center are queried.
tags={"walk": True , "building" : True}
point = (50.7717, 6.0841,)
size=500
dist = 5000
#OSMnx method geometries_from_point queryies OSM data using a point
gdf = ox.geometries_from_point(point, tags, dist=dist)
gdf_proj = ox.project_gdf(gdf)

NameError: name 'ox' is not defined

In [ ]:
gdf.info()

In [ ]:
gdf.head()

In [ ]:
#Quering OSM by using the defined point.
#This figure ground plan captures all the buildings or highway at a distance of 5km from city center.
bbox = ox.utils_geo.bbox_from_point(point=point, dist=dist, project_utm=True)
fp = f"./aachen_col_schwarzplan.png"
#Generating figure ground plan for the city of aachen and colouring all the buildings as red and everything in the background as white
fig, ax = ox.plot_footprints(
    gdf_proj,
    bbox=bbox,
    bgcolor="white",
    color="red",
    filepath=fp,
    dpi=300,
    save=True,
    show=False,
    close=True,
)
Image(fp, height=size, width=size)

In [ ]:
#Creating an interactive map for aachen using iphyleaflet and city center coordinates as a point of reference.
center = (50.7717, 6.0841)
map_aachen = Map(center=center, zoom=11)
map_aachen

In [ ]:
#Adding the GeoDataFrame to the map.
#GeodataFrame contains the figure ground plan of Jülich. So, adding the figure ground plan stored in 'gdf' into an interactive map with specific style defined below.
#geo_data = GeoData(geo_dataframe= gdf, style={'color': 'red', 'fillColor': '#3c758c', 'opacity':0.40, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6})
#map_aachen.add_layer(geo_data)


In [ ]:
#map_aachen

## (2) Searching for data on sustainable energy in and around the aachen

In [ ]:
#In this second part of the exercise, again the chosen city is aahen. 
#For this city, new data of renewable energies, particularly, wind turbines in and around the city of aachen are queried using OSM data.
#Further, these locations of wind turbines along with the location of city center are marked in the interactive map and few messages are added for these locations.

In [ ]:
#Now, for the renewable energies, wind turbine is selected as the type of generator method.
#Defining tag for this 'wind turbine' renewable energy. OSM data for all the wind turbines at a distance of 10km from city center are queried.
tag = {"generator:method" : "wind_turbine"}
#The value of point here is the aachen city center coordinates.
point = (50.7717, 6.0841)
dist = 5000
gdf2 = ox.geometries_from_point(point, tag, dist=dist)
gdf2.info()

In [ ]:
gdf2

In [ ]:
gdf3 = gdf2.copy()

In [ ]:
def extract(a):
    return (a.x, a.y)

In [ ]:
gdf3['geometry'] = gdf3['geometry'].apply(extract)

In [ ]:
gdf3.head(7)

In [ ]:
#Extracting all elements which are wind turbine and plotting them on the graph.
gdf2[gdf2['generator:method'] == "wind_turbine"].plot()

## (3) Visualize the data in an interactive map with the library Python Ipyleaflet

In [ ]:
#Setting up a map called aachen_turbine, with an appropriate zoom , for showing all the wind turbine locations on an interactive map.
aachen_windturbine = Map(center= point, zoom = 11)
geo_data = GeoData(geo_dataframe= gdf2, style={'color': 'red', 'fillColor': '#3c758c', 'opacity':0.20, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6})
aachen_windturbine.add_layer(geo_data)
aachen_windturbine

In [ ]:
#Adding another marker to locate the city center of aachen.
marker = Marker(location=center, draggable=False)
aachen_windturbine.add_layer(marker);
aachen_windturbine

In [ ]:
#aachenmitte = ox.geocode(center)
#windturbines = ox.geocode(windturbine_point)

In [ ]:
#Attention, getting the whole of Aachen takes many minutes, we will get the inner city only
G = ox.graph_from_point(point, dist=2000, network_type="walk")

In [ ]:
# Let us get out our graph from OSM first
ax, fig = ox.plot_graph(G, node_color='b', node_zorder=1, bgcolor='#FFFFFF')

In [ ]:
windturbine_locations = gdf3.geometry.values
routes = []
for index, windturbine_location in enumerate(windturbine_locations):
    #Set start and end point (value)
    start = ox.nearest_nodes(G, center[1], center[0])
    dest  = ox.nearest_nodes(G, windturbine_location[0], windturbine_location[1])

    #Calculate the shortest path between start and end point
    route = ox.shortest_path(G, start, dest)
    routes.append(route)

fig, ax = ox.plot_graph_routes(G, routes);
fig.savefig('world.jpg');